In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

# ------------------- Load and preprocess data -------------------

# Load label files
main_labels = pd.read_csv('data_labels_mainData.csv')
extra_labels = pd.read_csv('data_labels_extraData.csv')
combined_labels = pd.concat([main_labels, extra_labels], ignore_index=True)

# Image folder
image_folder = "C:/Users/golde/Downloads/Image_classification_data/patch_images"

# Task 1 splitting
train_labels_t1, temp_labels_t1 = train_test_split(
    combined_labels, test_size=0.3, random_state=42, stratify=combined_labels['isCancerous']
)
val_labels_t1, test_labels_t1 = train_test_split(
    temp_labels_t1, test_size=0.5, random_state=42, stratify=temp_labels_t1['isCancerous']
)

# Task 2 splitting
train_labels_t2, temp_labels_t2 = train_test_split(
    main_labels, test_size=0.3, random_state=42, stratify=main_labels['cellType']
)
val_labels_t2, test_labels_t2 = train_test_split(
    temp_labels_t2, test_size=0.5, random_state=42, stratify=temp_labels_t2['cellType']
)

# Helper to load images
def load_images(df, image_folder):
    images = []
    for img_name in df['ImageName']:
        img_path = os.path.join(image_folder, img_name)
        try:
            img = Image.open(img_path).convert("RGB")
            images.append(img)
        except Exception as e:
            print(f"Error loading {img_name}: {e}")
            images.append(None)
    return images

# Load Task 1 images
train_images_t1 = load_images(train_labels_t1, image_folder)
val_images_t1 = load_images(val_labels_t1, image_folder)
test_images_t1 = load_images(test_labels_t1, image_folder)

# Load Task 2 images
train_images_t2 = load_images(train_labels_t2, image_folder)
val_images_t2 = load_images(val_labels_t2, image_folder)
test_images_t2 = load_images(test_labels_t2, image_folder)

# Preprocess images
def preprocess_images(image_list):
    image_list = [img for img in image_list if img is not None]
    return np.array([np.asarray(img, dtype=np.float32) / 255.0 for img in image_list])

train_images_t1 = preprocess_images(train_images_t1)
val_images_t1 = preprocess_images(val_images_t1)
test_images_t1 = preprocess_images(test_images_t1)

train_images_t2 = preprocess_images(train_images_t2)
val_images_t2 = preprocess_images(val_images_t2)
test_images_t2 = preprocess_images(test_images_t2)

# Labels for Task 1
y_train_t1 = train_labels_t1['isCancerous'].values.astype(np.float32)
y_val_t1 = val_labels_t1['isCancerous'].values.astype(np.float32)
y_test_t1 = test_labels_t1['isCancerous'].values.astype(np.float32)

# Labels for Task 2
y_train_t2 = to_categorical(train_labels_t2['cellType'].values)
y_val_t2 = to_categorical(val_labels_t2['cellType'].values)
y_test_t2 = to_categorical(test_labels_t2['cellType'].values)

Task 1 (isCancerous) class balance:
Train: isCancerous
0    0.651451
1    0.348549
Name: proportion, dtype: float64
Val: isCancerous
0    0.651545
1    0.348455
Name: proportion, dtype: float64
Test: isCancerous
0    0.651216
1    0.348784
Name: proportion, dtype: float64

Task 2 (cellType) class balance:
Train: cellType
2    0.412155
1    0.256965
0    0.190847
3    0.140032
Name: proportion, dtype: float64
Val: cellType
2    0.412399
1    0.256739
0    0.190701
3    0.140162
Name: proportion, dtype: float64
Test: cellType
2    0.412121
1    0.257239
0    0.190572
3    0.140067
Name: proportion, dtype: float64


In [ ]:
# ------------------- Build and train CNN -------------------
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_t1), y=y_train_t1)
class_weights = dict(enumerate(class_weights))
print("📏 Class weights:", class_weights)

# Model definition
model_t1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(27, 27, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model_t1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(train_images_t1)

# Train
history = model_t1.fit(
    datagen.flow(train_images_t1, y_train_t1, batch_size=32),
    validation_data=(val_images_t1, y_val_t1),
    epochs=20,
    class_weight=class_weights,
    verbose=1
)


In [ ]:
# ------------------- Evaluation -------------------

# Accuracy & Loss Plots
def plot_training(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label="Train Acc")
    plt.plot(history.history['val_accuracy'], label="Val Acc")
    plt.title("Accuracy over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label="Train Loss")
    plt.plot(history.history['val_loss'], label="Val Loss")
    plt.title("Loss over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training(history)

In [ ]:
# Predictions
y_probs = (model_t1.predict(test_images_t1) >= 0.5).astype(int)


print(classification_report(y_test_t1, y_probs, target_names=["Not Cancerous", "Cancerous"]))

# Per-class bar plot
def plot_precision_recall_f1(y_true, y_pred, class_names):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred)
    metrics = {'Precision': precision, 'Recall': recall, 'F1-score': f1}

    plt.figure(figsize=(10, 5))
    for i, (metric, values) in enumerate(metrics.items()):
        plt.bar(np.arange(len(values)) + i*0.25, values, width=0.25, label=metric)
    
    plt.xticks(np.arange(len(class_names)) + 0.25, class_names)
    plt.ylim(0, 1.1)
    plt.ylabel("Score")
    plt.title("Per-Class Metrics")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_precision_recall_f1(y_test_t1, y_pred_final, ["Not Cancerous", "Cancerous"])